In [1]:
import glob

# Bead Data

In [8]:
def bead_path(viscosity, solute, size, magnification):
    path = "../data/beads/%s_visc_%s_%s_um-%s*"%(viscosity, solute, size, magnification)
    return glob.glob(path)

In [11]:
bead_path("13.2", "PVP", "0.4", "40")

['../data/beads/13.2_visc_PVP_0.4_um-40x',
 '../data/beads/13.2_visc_PVP_0.4_um-40x-2']

# Onion Data

In [10]:
onion_both = "../data/onion_1/Onion-cytoplasm-20x-3/*.bmp"
onion_tri = "../data/onion_2/Onion_bulk_20x/*.bmp"